Importación de Librerías

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

torch: Es la biblioteca principal de PyTorch que proporciona tensores y otras funcionalidades esenciales.
nn: Es el módulo de redes neuronales de PyTorch, que contiene todos los componentes para construir redes neuronales.
DataLoader: Clase que permite cargar los datos de manera eficiente en mini-lotes, con opciones de barajado y carga paralela.
datasets: Módulo que proporciona acceso a varios datasets comunes.
ToTensor: Transforma imágenes en formato PIL o numpy arrays a tensores de PyTorch.
Descarga del Dataset MNIST

In [ ]:
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)


datasets.MNIST: Clase para descargar y cargar el dataset MNIST.
root="data": Directorio donde se almacenarán los datos.
train=False/True: Indica si se deben cargar los datos de prueba (False) o de entrenamiento (True).
download=True: Descarga los datos si no están disponibles en el directorio especificado.
transform=ToTensor(): Aplica la transformación para convertir las imágenes a tensores de PyTorch.
Definición de DataLoader

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)


DataLoader: Clase que proporciona una manera conveniente de cargar datos en mini-lotes.
batch_size=64: Número de muestras por mini-lote.
shuffle=True/False: Mezcla los datos cada vez que se cargan (True para entrenamiento, False para prueba).
Imprimir la Forma de los Datos y los Targets

In [ ]:
print(training_data.data.shape)
print(training_data.targets.shape)


training_data.data.shape: Imprime la forma del tensor de datos de entrenamiento.
training_data.targets.shape: Imprime la forma del tensor de etiquetas de entrenamiento.
Definición de la Arquitectura de la Red Neuronal Mejorada

In [ ]:
class ImprovedNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


class ImprovedNeuralNetwork(nn.Module): Define la clase de la red neuronal, heredando de nn.Module.
init(self): Constructor de la clase.

super().__init__(): Llama al constructor de la clase base nn.Module.
self.flatten = nn.Flatten(): Define una capa que aplana las imágenes de 28x28 píxeles a un vector de 784 elementos.
self.linear_relu_stack = nn.Sequential(...): Define una secuencia de capas:
nn.Linear(28*28, 1024): Capa lineal con 784 entradas y 1024 salidas.
nn.BatchNorm1d(1024): Normalización por lotes para 1024 características, que ayuda a estabilizar y acelerar el entrenamiento.
nn.ReLU(): Función de activación ReLU.
nn.Dropout(0.5): Dropout con una tasa del 50%, que ayuda a prevenir el sobreajuste.
nn.Linear(1024, 512): Capa lineal con 1024 entradas y 512 salidas.
nn.BatchNorm1d(512): Normalización por lotes para 512 características.
nn.ReLU(): Función de activación ReLU.
nn.Dropout(0.5): Dropout con una tasa del 50%.
nn.Linear(512, 256): Capa lineal con 512 entradas y 256 salidas.
nn.BatchNorm1d(256): Normalización por lotes para 256 características.
nn.ReLU(): Función de activación ReLU.
nn.Linear(256, 10): Capa lineal con 256 entradas y 10 salidas, correspondiente a las 10 clases del dataset MNIST.
forward(self, x): Método que define el flujo de datos a través de la red.

x = self.flatten(x): Aplana la imagen de entrada.
logits = self.linear_relu_stack(x): Pasa los datos a través de la secuencia de capas.
return logits: Devuelve los logits (salidas no normalizadas de la red).
Preparación del Modelo y el Dispositivo

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


loss_fn: Define la función de pérdida como la pérdida de entropía cruzada, adecuada para problemas de clasificación.
optimizer: Define el optimizador Adam:
model.parameters(): Pasa los parámetros del modelo al optimizador.
lr=1e-3: Establece la tasa de aprendizaje a 0.001.
Scheduler para Disminuir el Learning Rate

In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


scheduler: Define un scheduler para ajustar el learning rate:
step_size=5: Disminuye el learning rate cada 5 épocas.
gamma=0.1: Factor por el que se multiplica el learning rate (0.1).
Función de Entrenamiento

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_num, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_num % 100 == 0:
            loss, current = loss.item(), (batch_num + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


train(dataloader, model, loss_fn, optimizer): Función para entrenar el modelo.
size = len(dataloader.dataset): Obtiene el tamaño del dataset.
model.train(): Pone el modelo en modo de entrenamiento.
for batch_num, (X, y) in enumerate(dataloader): Itera sobre los mini-lotes del dataloader.
X, y = X.to(device), y.to(device): Mueve los datos y las etiquetas al dispositivo seleccionado.
pred = model(X): Realiza predicciones para el mini-lote.
loss = loss_fn(pred, y): Calcula la pérdida entre las predicciones y las etiquetas reales.
optimizer.zero_grad(): Resetea los gradientes de los parámetros del modelo.
loss.backward(): Calcula los gradientes de la pérdida con respecto a los parámetros del modelo.
optimizer.step(): Actualiza los parámetros del modelo según los gradientes.
if batch_num % 100 == 0: Cada 100 lotes, imprime el progreso.

Todo el código junto ahora.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Descarga el dataset MNIST
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Define el dataloader
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

# Imprime la forma de los datos y los targets
print(training_data.data.shape)
print(training_data.targets.shape)

# Define la arquitectura de la red neuronal mejorada
class ImprovedNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = ImprovedNeuralNetwork()
device = (
    "cuda" if torch.cuda.is_available() 
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

model = model.to(device)
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Scheduler para disminuir el learning rate
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_num, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_num % 100 == 0:
            loss, current = loss.item(), (batch_num + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    accuracy = correct * 100
    print(f"Test Error: \n Accuracy: {accuracy:>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return accuracy, test_loss

# Implementar Early Stopping
best_accuracy = 0
epochs = 20
patience = 3
trigger_times = 0

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    accuracy, avg_loss = test(test_dataloader, model, loss_fn)
    
    # Actualizar el scheduler
    scheduler.step()
    
    # Early Stopping
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        trigger_times = 0
    else:
        trigger_times += 1
        print(f'Early stopping trigger times: {trigger_times}')
        
        if trigger_times >= patience:
            print('Early stopping!')
            break

print(f"Best Accuracy: {best_accuracy:>0.1f}%")
print("Done!")



torch.Size([60000, 28, 28])
torch.Size([60000])
Using cpu device
ImprovedNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=1024, out_features=512, bias=True)
    (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=512, out_features=256, bias=True)
    (9): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Linear(in_features=256, out_features=10, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 2.322450  [   64/60000]
loss: 0.518416  [ 6464/60000]
loss: 0.364613  [12864/60000]
loss: 0.254473  [19264/60000]
loss: 0.265586  [

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Descarga el dataset MNIST
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Define el dataloader
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

# Imprime la forma de los datos y los targets
print(training_data.data.shape)
print(training_data.targets.shape)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
device = (
    "cuda" if torch.cuda.is_available() 
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

model = model.to(device)
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_num, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_num % 100 == 0:
            loss, current = loss.item(), (batch_num + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


torch.Size([60000, 28, 28])
torch.Size([60000])
Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 2.297163  [   64/60000]
loss: 2.294877  [ 6464/60000]
loss: 2.298019  [12864/60000]
loss: 2.292419  [19264/60000]
loss: 2.283015  [25664/60000]
loss: 2.285678  [32064/60000]
loss: 2.276286  [38464/60000]
loss: 2.271629  [44864/60000]
loss: 2.258700  [51264/60000]
loss: 2.259532  [57664/60000]
Test Error: 
 Accuracy: 44.4%, Avg loss: 2.254423 

Epoch 2
-------------------------------
loss: 2.258667  [   64/60000]
loss: 2.249626  [ 6464/60000]
loss: 2.237247  [12864/60000]
loss: 2.236071  [19264/60000]
loss: 2.232283  [25664/60000]
loss: 2.222603  [32064/60000]
loss

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Descarga el dataset MNIST
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Define el dataloader
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

# Imprime la forma de los datos y los targets
print(training_data.data.shape)
print(training_data.targets.shape)

# Define la arquitectura de la red neuronal mejorada
class ImprovedNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = ImprovedNeuralNetwork()
device = (
    "cuda" if torch.cuda.is_available() 
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

model = model.to(device)
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_num, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_num % 100 == 0:
            loss, current = loss.item(), (batch_num + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

torch.Size([60000, 28, 28])
torch.Size([60000])
Using cpu device
ImprovedNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): ReLU()
    (8): Linear(in_features=256, out_features=10, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 2.307417  [   64/60000]
loss: 0.409956  [ 6464/60000]
loss: 0.304417  [12864/60000]
loss: 0.256013  [19264/60000]
loss: 0.101194  [25664/60000]
loss: 0.135611  [32064/60000]
loss: 0.103361  [38464/60000]
loss: 0.230500  [44864/60000]
loss: 0.051359  [51264/60000]
loss: 0.082579  [57664/60000]
Test Error: 
 Accuracy: 95.8%, Avg loss: 0.138241 

Epoch 2
-------------------------------
loss: 0.186645  [ 

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Definimos las transformaciones para preprocesado de las imágenes
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Normalización con media y desviación estándar de MNIST
])

# Cargamos el dataset MNIST con las transformaciones definidas
train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

# Creamos los DataLoaders para los conjuntos de entrenamiento y prueba
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

from torch.nn import functional as F

# Definimos la arquitectura de la red neuronal convolucional mejorada
class ImprovedCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)   # Primera capa convolucional
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # Segunda capa convolucional
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # Tercera capa convolucional
        self.pool = nn.MaxPool2d(2, 2)  # Capa de Max Pooling
        self.dropout = nn.Dropout(0.25) # Capa de Dropout para regularización
        self.fc1 = nn.Linear(3 * 3 * 128, 256) # Primera capa totalmente conectada
        self.fc2 = nn.Linear(256, 128)         # Segunda capa totalmente conectada
        self.fc3 = nn.Linear(128, 10)          # Capa de salida

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # Conv1 -> ReLU -> Pooling
        x = self.pool(F.relu(self.conv2(x))) # Conv2 -> ReLU -> Pooling
        x = self.pool(F.relu(self.conv3(x))) # Conv3 -> ReLU -> Pooling
        x = self.dropout(x)                  # Dropout
        x = x.view(-1, 3 * 3 * 128)          # Aplanamiento tras las capas convolucionales
        x = F.relu(self.fc1(x))              # FC1 -> ReLU
        x = self.dropout(x)                  # Dropout
        x = F.relu(self.fc2(x))              # FC2 -> ReLU
        x = self.fc3(x)                      # FC3 (salida)
        return F.log_softmax(x, dim=1)       # Log Softmax para usar con entropía cruzada

# Instanciamos la red neuronal mejorada
model = ImprovedCNN()
criterion = nn.CrossEntropyLoss() # Definimos la función de pérdida
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) # Definimos el optimizador Adam

# Seleccionamos el dispositivo (GPU si está disponible, sino CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # Movemos el modelo al dispositivo

# Entrenamiento del modelo
model.train()
num_epochs = 10

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device) # Movemos datos y etiquetas al dispositivo

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels) # Calculamos la pérdida

        # Backward pass y optimización
        optimizer.zero_grad() # Limpiamos los gradientes
        loss.backward()       # Retropropagamos la pérdida
        optimizer.step()      # Actualizamos los parámetros del modelo

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Evaluación del modelo
model.eval() # Ponemos el modelo en modo evaluación

correct = 0
total = 0
with torch.no_grad(): # Desactivamos el cálculo de gradientes
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1) # Predicción de la clase con mayor probabilidad
        total += labels.size(0)
        correct += (predicted == labels).sum().item() # Contamos los aciertos

accuracy = 100 * correct / total # Calculamos la precisión
print(f'Accuracy of the network on the 10000 test images: {accuracy:.2f}%')
